In [1]:
import boto3
import pandas as pd
from pyathena import connect
import uuid
from datetime import datetime, date,timedelta
import requests
import psycopg2
import re
import json
import numpy as np
import nltk
from urlextract import URLExtract
from nltk.stem import WordNetLemmatizer 
from collections import Counter

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns   
import nltk
import re
import csv
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from wordcloud import WordCloud
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
pwd

'C:\\Users\\Asim\\Desktop\\Baby Destination Data\\final_model\\sub_category\\python_script'

#### Importing the data files for lifestage training

In [6]:
training_path = "C:\\Users\\Asim\\Desktop\\Baby Destination Data\\final_model\\sub_category\\training_data\\"

In [7]:
merged_file = pd.read_csv(training_path+"final_sub_category_traininig_data.csv")

In [8]:
merged_file.shape

(24848, 2)

In [9]:
merged_file.groupby('sub_category').count()

,text
sub_category,
Learning,2679
breastfeeding,2238
cold_and_cough,2382
nutrition,2852
oral_care,2259
others,2248
pregnancy,2534
recipes,2130
skin_care,2833


In [10]:
#merged_file = merged_file[merged_file['text'].apply(lambda x:len(x)>50)].reset_index(drop=True)
merged_file.drop_duplicates(subset='text',inplace=True)

In [11]:
merged_file.groupby('sub_category').count()

,text
sub_category,
Learning,2679
breastfeeding,2238
cold_and_cough,2382
nutrition,2852
oral_care,2259
others,2248
pregnancy,2534
recipes,2130
skin_care,2833


#### Preprocessing

In [15]:
stop_words = pd.read_csv('stop_words_model_text.txt')
stopWordSet = [x for x in stop_words['stop']]
stopWordSet = list(set(stopWordSet))
len(stopWordSet)

4072

In [17]:
def stopWords(text):
    text = text.split()
    text = [word for word in text if word not in stopWordSet]
    text = ' '.join(text)
    return(text)

In [18]:
def url_extractor(text):
    extractor = URLExtract()
    urls = extractor.find_urls(text)
    text = text.split(' ')
    text = [ words for words in text if words not in urls]
    text = ' '.join(text)
    return(text)

In [19]:
def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split(' ')
    text = [lemmatizer.lemmatize(term) for term in text]
    text = ' '.join(text)
    return(text)

In [26]:
def preprocessing_text(text):
    text = str(text).lower()
    text = re.sub('\n','',text)
    text = url_extractor(text)
    text = re.sub('[^A-Za-z0-9]+', ' ',text)
    text = stopWords(text)
    text = lemmatizer(text)
    text = re.sub('[0-9]','',text)
    return(text)

#### Applying the preprocessing on the file

In [31]:
merged_file['text'] = merged_file['text'].apply(lambda x: preprocessing_text_text(x))

In [32]:
data_file = merged_file

#### Using the Wordnet Lemmatizer

#### Splitting the dataset into training and testing 

In [33]:
x_train,x_test,y_train,y_test = model_selection.train_test_split(data_file['text'],data_file['sub_category'], test_size = 0.1, random_state=1)

#### Label Encoder for the text dataset 

In [34]:
label_encoder = LabelEncoder()
label_train = label_encoder.fit_transform(y_train)
label_test = label_encoder.fit_transform(y_test)

In [35]:
len(label_train),len(label_test)

(22363, 2485)

#### TFidf vectorizer experiment

In [36]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3),max_features=18000)
tfidf_vect_ngram.fit(data_file['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

#### Using the model algoritms 

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,consensus_score

In [38]:
naive_bayes  = MultinomialNB()
naive_bayes.fit(xtrain_tfidf_ngram,label_train)
predictions = naive_bayes.predict(xtest_tfidf_ngram)
accuracy_score(predictions,label_test)

0.7851106639839034

#### Best Hyperparameter Tuning of RandomForest

In [39]:
forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=125, n_jobs=-1,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [40]:
forest.fit(xtrain_tfidf_ngram,label_train)
predictions = forest.predict(xtest_tfidf_ngram)
accuracy_score(predictions,label_test)

0.8370221327967807

In [45]:
forest.oob_score_

0.8365603899297948

### Dumping sub_category models, vectorizers and encoders 

In [42]:
#Dumping the model 
import joblib
models_path = "C:\\Users\\Asim\\Desktop\\Baby Destination Data\\final_model\\sub_category\\models\\"
model_filename ='sub_category_model_joblib.pkl'
joblib.dump(forest,open(models_path+model_filename, 'wb'))

In [43]:
#Dumping the vectorizer
vectorizer_filename = 'sub_category_vectorizer_joblib.pkl'
joblib.dump(tfidf_vect_ngram,open(models_path+vectorizer_filename,'wb'))

In [44]:
#Dumping the Label Encoder 
encoder_filename = 'sub_category_encoder_joblib.pkl'
joblib.dump(label_encoder,open(models_path+encoder_filename,'wb'))

#### Loading_models,  vectorizers and encoders

In [17]:
import pickle

In [18]:
model_filename ='sub_category_model.pkl'
vectorizer_filename = 'sub_category_vectorizer.pkl'
encoder_filename = 'sub_category_encoder.pkl'

In [19]:
models_path = "C:\\Users\\Asim\\Desktop\\Baby Destination Data\\final_model\\sub_category\\models\\"
with open(models_path+model_filename,'rb') as f:
    model_sub_category = pickle.load(f)

In [20]:
with open(models_path+vectorizer_filename,'rb') as f:
    vectorizer_sub_category = pickle.load(f)

In [21]:
with open(models_path+encoder_filename,'rb') as f:
    encoder_sub_category = pickle.load(f)

#### Prediction on base of single text for checking model 

In [22]:
text ="""
Hi mam how  are you thank you for your support i want to ask u that i m suffering from itching in vagina outer skin can u suggest me plz

"""

In [24]:
text = preprocessing_text(text)
text_tfidf = vectorizer_sub_category.transform([text])

In [25]:
prediction = model_sub_category.predict(text_tfidf)

In [26]:
text

'support suffering from itching vagina outer suggest'

In [27]:
model_sub_category.predict_proba(text_tfidf)

array([[0.25956638, 0.08138839, 0.14710296, 0.08549221, 0.03757381,
        0.17216508, 0.06511106, 0.01829956, 0.06990692, 0.06339364]])

In [28]:
encoder_sub_category.classes_

array(['Learning', 'breastfeeding', 'cold_and_cough', 'nutrition',
       'oral_care', 'others', 'pregnancy', 'recipes', 'skin_care',
       'women_health_skin'], dtype=object)

In [246]:
encoder_sub_category.inverse_transform(prediction)

array(['pregnancy'], dtype=object)

#### THE END

#### Confusion Matrix and see where the model is making errors 

In [98]:
confusion_matrix(label_test,predictions)

array([[488,   6,  88,   1,   2,   2,   2,   5],
       [ 52, 462,  68,   3,   1,  19,  16,   2],
       [ 49,  32, 590,  10,   1,  20,   9,   6],
       [  5,   7,  24, 530,   2,   4,   6,   2],
       [  6,   8,  15,   1, 464,   1,   5,  12],
       [ 13,   4,  15,   0,   0, 588,   2,   5],
       [  4,   8,  29,   1,   2,   7, 480,   1],
       [ 32,  25,  93,   0,  31,  11,  25, 264]], dtype=int64)

In [99]:
label_encoder.classes_

array(['breastfeeding', 'cold_and_cough', 'nutrition', 'oral_care',
       'pregnancy', 'recipes', 'skin_care', 'women_health_skin'],
      dtype=object)

In [103]:
test_text = pd.DataFrame()
test_text['text'] = x_test
test_text.reset_index(drop=True,inplace=True)
prediction_dataframe = pd.DataFrame()
prediction_dataframe['prediction'] = predictions
prediction_dataframe['real_labels'] = label_test
prediction_dataframe['text'] = test_text['text']

In [104]:
unmatched_labels = prediction_dataframe.where((prediction_dataframe['prediction'] != prediction_dataframe['real_labels']))
unmatched_labels = unmatched_labels[unmatched_labels['prediction'].notnull()]
unmatched_labels.reset_index(drop=True,inplace=True)

In [105]:
unmatched_labels['prediction'] = unmatched_labels['prediction'].apply(lambda x:int(x))
unmatched_labels['real_labels'] = unmatched_labels['real_labels'].apply(lambda x:int(x))
unmatched_labels['prediction_text'] = label_encoder.inverse_transform(unmatched_labels['prediction'])
unmatched_labels['real_label_text'] = label_encoder.inverse_transform(unmatched_labels['real_labels'])

In [106]:
unmatched_labels[:10]

,prediction,real_labels,text,prediction_text,real_label_text
0,2,7,jeera or sauff watar banane ke baad jo jeera s...,nutrition,women_health_skin
1,2,0,tejaswani bhambota pankaj,nutrition,breastfeeding
2,0,1,hii mommiesi had lump fever with chill unbeara...,breastfeeding,cold_and_cough
3,2,6,usko neend ati hogi u time ji time wo itching ...,nutrition,skin_care
4,2,0,or 15 min 20 min shoulder pr rkhna is not poss...,nutrition,breastfeeding
5,0,2,1 of my twin had same issue when they wer same...,breastfeeding,nutrition
6,2,7,kya pure din garam pani pine se constipation h...,nutrition,women_health_skin
7,0,1,my 10 week old ha some nasal congestion and mi...,breastfeeding,cold_and_cough
8,4,7,bsimhairfallbreastfeeding or not hairfall is c...,pregnancy,women_health_skin
9,5,2,aaj avi lunch me m ne baby ke liye or apne liy...,recipes,nutrition


In [107]:
label_encoder.classes_

array(['breastfeeding', 'cold_and_cough', 'nutrition', 'oral_care',
       'pregnancy', 'recipes', 'skin_care', 'women_health_skin'],
      dtype=object)

In [108]:
unmatched_labels.to_csv(training_path+'unmatched_labels.csv',index=False)

#### Checking the model 

In [42]:
#test_path = "C:\\Users\\Asim\\Desktop\\Tay Keith\\lifestage\\model_checking\\"

In [43]:
testing_data = pd.read_csv('predictions_on_research_file.csv')

In [44]:
testing_data[:4]

,text,groupname,prediction,check
0,my lo is 1and half month old c section deliver...,Breastfeeding Support For Indian Mothers,['mother'],0.0
1,my son is a week shy of 4months and is ebf he ...,Breastfeeding Support For Indian Mothers,['mother'],1.0
2,beautiful to read,Breastfeeding Support For Indian Mothers,['baby'],0.0
3,bsimlaugh,Breastfeeding Support For Indian Mothers,['baby'],0.0


In [46]:
testing_data['text'] = testing_data['text'].apply(lambda x:preprocessing(str(x)))
testing_data['text'] = testing_data['text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(term) for term in x.split()))

In [47]:
testing_data.shape

(1996, 4)

In [48]:
prediction_file = []
for text in testing_data['text']:
    prediction_file.append(forest.predict(tfidf_vect_ngram.transform([text])))

In [49]:
prediction_file

[array([0]),
 array([0]),
 array([2]),
 array([4]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),

In [50]:
len(prediction_file),testing_data.shape

(1996, (1996, 4))

In [51]:
testing_data['sub_category_prediction'] = prediction_file

In [52]:
testing_data['sub_category_prediction'] = testing_data['sub_category_prediction'].apply(lambda x:label_encoder.inverse_transform(x))

In [53]:
testing_data[:10]

,text,groupname,prediction,check,sub_category_prediction
0,my lo is 1and half month old c section deliver...,Breastfeeding Support For Indian Mothers,['mother'],0.0,[breastfeeding]
1,my son is a week shy of 4months and is ebf he ...,Breastfeeding Support For Indian Mothers,['mother'],1.0,[breastfeeding]
2,beautiful to read,Breastfeeding Support For Indian Mothers,['baby'],0.0,[nutrition]
3,bsimlaugh,Breastfeeding Support For Indian Mothers,['baby'],0.0,[skin_care]
4,hi mommy my lo is 12 day shy of 5th month she ...,Breastfeeding Support For Indian Mothers,['mother'],0.0,[breastfeeding]
5,my lo is 5 month old she is on ebf we are plan...,Breastfeeding Support For Indian Mothers,['baby'],0.0,[breastfeeding]
6,hello mommy i need help for my friend she deli...,Breastfeeding Support For Indian Mothers,['mother'],1.0,[breastfeeding]
7,here my little j is 18 month is now on new mil...,Breastfeeding Support For Indian Mothers,['baby'],1.0,[breastfeeding]
8,mommy today i went to my doc for 6 week vaccin...,Breastfeeding Support For Indian Mothers,['mother'],0.0,[breastfeeding]
9,hi momies i have to join office starting janua...,Breastfeeding Support For Indian Mothers,['mother'],1.0,[breastfeeding]


In [54]:
testing_data.to_csv('sub_category_predictions_on_research_file.csv',index=False)

#### Model Prediction Code snippets on the single text data 

In [395]:
text  = """
meri bati 5yrs ki 4 5 din se daily uske forehead pe bich me halki swelling aa jati h koi batayega aisa kyu ho raha h plz suggest me nd advance me thank you
"""

In [396]:
text = preprocessing(text)

In [397]:
text

'meri bati 5yrs ki 4 5 din se daily uske forehead pe bich me halki swelling aa jati h koi batayega aisa kyu ho raha h plz suggest me nd advance me thank you'

In [398]:
text_tfidf = tfidf_vect_ngram.transform([text])

In [399]:
forest.predict_proba(text_tfidf)

array([[0.02842371, 0.12364546, 0.11919443, 0.05954996, 0.04368642,
        0.03334799, 0.5210652 , 0.07108684]])

In [400]:
forest.predict(text_tfidf)

array([6])

In [401]:
label_encoder.classes_

array(['breastfeeding', 'cold_and_cough', 'nutrition', 'oral_care',
       'pregnancy', 'recipes', 'skin_care', 'women_health_skin'],
      dtype=object)

In [402]:
pwd

'C:\\Users\\Asim\\Desktop\\Tay Keith\\models\\sub_category\\model_building\\training_data_gather'

array([[ 106,   86,    4,    0,    2,    0,    0,    4],
       [   6, 1044,   32,    2,    8,    0,   12,   17],
       [   5,  158,  209,    1,   11,    3,   12,    4],
       [   0,   50,    3,   40,    2,    0,    0,    1],
       [   1,  109,   11,    4,  146,    0,    6,    1],
       [   0,   16,    7,    0,    1,   12,    3,    0],
       [   4,   99,   14,    2,   13,    0,  129,    0],
       [   2,  216,    1,    0,    0,    0,    4,  143]], dtype=int64)

In [295]:
label_encoder.classes_

array(['big_kid', 'infant', 'mother', 'none', 'others', 'pre_pregnancy',
       'pregnancy', 'toddler'], dtype=object)

In [146]:
test_text = pd.DataFrame()
test_text['text'] = x_test
test_text.reset_index(drop=True,inplace=True)
prediction_dataframe = pd.DataFrame()
prediction_dataframe['prediction'] = prediction
prediction_dataframe['real_labels'] = label_test
prediction_dataframe['text'] = test_text['text']

In [147]:
prediction_dataframe[:5]

,prediction,real_labels,text
0,4,4,what a rubbish thinking life is your aj ke tim...
1,1,1,is it breast feeding baby if it is so nothing ...
2,1,1,change ur dr 9 kg ur baby s wt is good don t w...
3,0,0,try the homemade no sugar cooky cereal mix and...
4,1,7,mahak singh make aata dried fruit laddus bake ...


In [148]:
unmatched_labels = prediction_dataframe.where((prediction_dataframe['prediction'] != prediction_dataframe['real_labels']))

In [149]:
unmatched_labels = unmatched_labels[unmatched_labels['prediction'].notnull()]

In [150]:
unmatched_labels.reset_index(drop=True,inplace=True)

In [151]:
unmatched_labels['prediction'] = unmatched_labels['prediction'].apply(lambda x:int(x))
unmatched_labels['real_labels'] = unmatched_labels['real_labels'].apply(lambda x:int(x))
unmatched_labels['prediction_text'] = label_encoder.inverse_transform(unmatched_labels['prediction'])
unmatched_labels['real_label_text'] = label_encoder.inverse_transform(unmatched_labels['real_labels'])

In [152]:
unmatched_labels[:10]

,prediction,real_labels,text,prediction_text,real_label_text
0,1,7,mahak singh make aata dried fruit laddus bake ...,infant,toddler
1,1,4,yr aj ke jamane m to 1 hi baby bhot hai besa b...,infant,others
2,1,7,due the acid reflux kid cont degest the food,infant,toddler
3,1,2,you can try mystical hair oil and mask by pure...,infant,mother
4,1,2,hi mommy hav been posting and v happy to get p...,infant,mother
5,1,7,bate krke ya mind divert krke khilao ajkl khil...,infant,toddler
6,1,7,stuff veggie in paratha or chapati make a doug...,infant,toddler
7,0,1,anyone tried suvarnaprashan ayurvedic immuniza...,big_kid,infant
8,1,2,aur baby frequently pita hai to din bhar baith...,infant,mother
9,1,2,guy you all are awesome thanks for wonderful s...,infant,mother


In [128]:
label_encoder.classes_

array(['big_kid', 'infant', 'mother', 'none', 'others', 'pre_pregnancy',
       'pregnancy', 'toddler'], dtype=object)

In [154]:
unmatched_labels.to_csv('unmatched_labels.csv',index=False)

In [55]:
pwd

'C:\\Users\\Asim\\Desktop\\Tay Keith\\models\\sub_category\\model_building'

In [57]:
df = pd.read_csv('refined_crawled_11_groups_data.csv')

In [58]:
df_1 = pd.read_csv('sub_category_predictions_on_research_file.csv')

In [ ]:
df['rawtext'] = df['rawtext'].aply(lambda x:)

In [18]:
df = pd.read_csv('refined_crawled_11_groups_data_new.csv',encoding='latin-1')

In [20]:
df.groupby('Sub_Category').count()

,groupname,rawtext
Sub_Category,,
bre\,1,1
breastfeeding,431,431
cold_and_cough,73,73
dha,4,4
ears),8,8
gear,2,2
gears,7,7
home_appliances,1,1
hygeine,13,13


In [403]:
pwd

'C:\\Users\\Asim\\Desktop\\Tay Keith\\models\\sub_category\\model_building\\training_data_gather'

In [36]:
learning = merged_file[merged_file['sub_category']=='Learning']

In [ ]:
support suffering from itching vagina outer suggest

In [50]:
learning[learning['text'].str.contains("itch")]

,sub_category,text
17671,Learning,"same was with my daughter,she had small bumps ..."
17972,Learning,"true my baby want to play dustbin, charger rem..."
18081,Learning,hello moms this is my first post meri beti ka ...
18241,Learning,if u hav top then get contrast color satin clo...
18262,Learning,reema sameer the thirsty crow story in english...
18291,Learning,"mry bety ko engaged krna bht mushkil h , bht t..."
18409,Learning,even my son is 2.5 he still enjoys all these.....
18417,Learning,hi mommies. yesterday night i observed pinworm...
18635,Learning,mere baby ka ek v toy shi salamt nhi milega ap...
18781,Learning,main thing kitchen se bartan or pani to missin...
